###  Connect to Your Workspace
#### The first thing you need to do is to connect to your workspace using the Azure ML SDK.
- In the azure portal create a workspace
- After creating the workspace open ML studio with the workspace and create a compute instance
- In the compute instance open the jupyter notebook
- open new python file
- connect to the workspace you created using the below code


In [1]:
import azureml.core
from azureml.core import Workspace
ws=Workspace.from_config()
print(azureml.core.VERSION,ws.name)

1.22.0 workspace19


### Create an Azure ML experiment in your workspace

In [2]:
from azureml.core import Model,Experiment
exp=Experiment(workspace=ws,name='iris-experiment')
print(exp.name)

iris-experiment


### Train and Register a Model
#### import required libraries for training and registering the model

In [3]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#### Start an interactive session using the below code

In [4]:
run=exp.start_logging()

#### load the data from the specific folder 

In [31]:
print('loading data.......................')
data=pd.read_csv('data/IRIS.csv')
data.head()

loading data.......................


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


#### convert the target labels from string to int using label encoding

In [6]:
# Import label encoder 
from sklearn import preprocessing 

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

# Encode labels in column 'species'. 
data['species']= label_encoder.fit_transform(data['species']) 

data['species'].unique() 

array([0, 1, 2])

#### # Split data into training set and test set

In [32]:
x,y=data[['sepal_length','sepal_width','petal_length','petal_width']].values,data['species'].values
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

#### Train a decision tree model


In [10]:

print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
 

Training a decision tree model
Accuracy: 0.9777777777777777


#### Evalute the model

In [11]:
auc = roc_auc_score(y_test,y_scores,multi_class="ovr",average="weighted")
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

AUC: 0.9852941176470588


#### Save the trained model as a pkl file a specific directory and complete the run

In [ ]:

model_file = 'iris_model2.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)


run.complete()

#### Register the model in azure ml studio for future use 

In [13]:

run.register_model(model_path='outputs/iris_model2.pkl', model_name='iris_model2',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

Model trained and registered.


### Deploy a Model as a Web Service

Deploying an Azure Machine Learning model as a web service creates a REST API endpoint. You can send data to this endpoint and receive the prediction returned by the model.

You create a web service when you deploy a model to your local environment, Azure Container Instances, Azure Kubernetes Service, or field-programmable gate arrays (FPGA). You retrieve the URI used to access the web service by using the Azure Machine Learning SDK. If authentication is enabled, you can also use the SDK to get the authentication keys or tokens.

When you deploy a model, a Webservice object is returned with information about the service.

First, let's determine what models you have registered in the workspace.

In [34]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

iris_model2 version: 1
	 Training context : Inline Training
	 AUC : 0.9852941176470588
	 Accuracy : 0.9777777777777777




#### Right, now let's get the model that we want to deploy. By default, if we specify a model name, the latest version will be returned.

In [15]:
model = ws.models['iris_model2']
print(model.name, 'version', model.version)

iris_model2 version 1


#### We're going to create a web service to host this model, and this will require some code and configuration files; so let's create a folder for those.

In [16]:
import os

folder_name = 'iris_service2'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_iris.py")

iris_service2 folder created.


The web service where we deploy the model will need some Python code to load the input data, get the model from the workspace, and generate and return predictions. We'll save this code in an entry script (often called a scoring script) that will be deployed to the web service:

In [17]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('iris_model2')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['Iris-sectosa', 'Iris-versicolor','Iris-verginica']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Writing ./iris_service2/score_iris.py


The web service will be hosted in a container, and the container will need to install any required Python dependencies when it gets initialized. In this case, our scoring code requires scikit-learn, so we'll create a .yml file that tells the container host to install this into the environment.

In [18]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')

# Save the environment config as a .yml file
env_file = os.path.join(experiment_folder,"iris_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in ./iris_service2/iris_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



Now you're ready to deploy. We'll deploy the container a service named diabetes-service. The deployment process includes the following steps:

- Define an inference configuration, which includes the scoring and environment files required to load and use the model.
- Define a deployment configuration that defines the execution environment in which the service will be hosted. In this case, an Azure Container Instance.
- Deploy the model as a web service.
- Verify the status of the deployed service.

Deployment will take some time as it first runs a process to create a container image, and then runs a process to create a web service based on the image. When deployment has completed successfully, you'll see a status of Healthy.

In [19]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "iris-service2"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


Hopefully, the deployment has been successful and you can see a status of Healthy. If not, you can use the following code to check the status and get the service logs to help you troubleshoot.

In [36]:
print(service.state)
#print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

Healthy



Take a look at your workspace in Azure ML Studio and view the Endpoints page, which shows the deployed services in your workspace.

You can also retrieve the names of web services in your workspace by running the following code:

In [37]:
for webservice_name in ws.webservices:
    print(webservice_name)

iris-service2


Use the Web Service
With the service deployed, now you can consume it from a client application.

In [41]:
import json

x_new = [[6.7,3.9,5.2,5.3]]
print ('flower : {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})
print(input_json)

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)
print(predictions)
# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

flower : [6.7, 3.9, 5.2, 5.3]
{"data": [[6.7, 3.9, 5.2, 5.3]]}
["Iris-verginica"]
Iris-verginica


In [43]:
import json

# This time our input is an array of two feature arrays
x_new = [[3.7,1.0,5.2,2.3],
        [1.1,3.5,0.4,0.2]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [3.7, 1.0, 5.2, 2.3] Iris-verginica
Patient [1.1, 3.5, 0.4, 0.2] Iris-sectosa


The code above uses the Azure ML SDK to connect to the containerized web service and use it to generate predictions from your iris classification model. In production, a model is likely to be consumed by business applications that do not use the Azure ML SDK, but simply make HTTP requests to the web service.

Let's determine the URL to which these applications must submit their requests:

In [28]:

endpoint = service.scoring_uri
print(endpoint)

http://73e4ec73-6002-4464-8c50-a0a893f17628.southeastasia.azurecontainer.io/score


Now that you know the endpoint URI, an application can simply make an HTTP request, sending the patient data in JSON (or binary) format, and receive back the predicted class(es).

In [29]:
import requests
import json

x_new = [[6.7,3.0,5.2,2.3],
        [5.1,3.5,1.4,0.2]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [6.7, 3.0, 5.2, 2.3] Iris-verginica
Patient [5.1, 3.5, 1.4, 0.2] Iris-sectosa


You've deployed your web service as an Azure Container Instance (ACI) service that requires no authentication. This is fine for development and testing, but for production you should consider deploying to an Azure Kubernetes Service (AKS) cluster and enabling authentication. This would require REST requests to include an Authorization header.